In [25]:

#dump face data from database
#routeName: 公車路線
#direction: 公車方向
#date: 抓取特定日期後的資料, format: YYYY-MM-DD
#輸入：routeName, direction, date
#輸出：符合搜尋條件的臉部資料
def dump(routeName, direction, date = '2017-01-01'):
    
    print("Connecting......")

    import boto3, sys
    client = boto3.client('cognito-idp', region_name='ap-northeast-1')
    authResult = client.initiate_auth(AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': 'b04705007', 'PASSWORD': 'KIBANA!0856asdf'},
    ClientId='2on0keh94h3u7jn5b7oci65d2u')['AuthenticationResult']
    client = boto3.client('cognito-identity',
    region_name='ap-northeast-1')
    identityId =client.get_id(IdentityPoolId='ap-northeast-1:daed3759-c089-4347-8f1b-1639ae65e03c',
    Logins={'cognito-idp.ap-northeast-1.amazonaws.com/ap-northeast-1_dBMEa9aFc': authResult['IdToken']})['IdentityId']
    credentials =client.get_credentials_for_identity(IdentityId=identityId,
    Logins={'cognito-idp.ap-northeast-1.amazonaws.com/ap-northeast-1_dBMEa9aFc': authResult['IdToken']})['Credentials']
    from requests_aws4auth import AWS4Auth
    awsauth = AWS4Auth(credentials['AccessKeyId'],
    credentials['SecretKey'], 'ap-northeast-1', 'es',
    session_token=credentials['SessionToken'])
    from elasticsearch import Elasticsearch, RequestsHttpConnection
    es_client = Elasticsearch(
     hosts=[{'host':
    'search-prod-po-prodfa-rsq1nqydqzos-vhhea2hvm5zikrjmqb7h67zt4u.ap-northeast-1.es.amazonaws.com', 'port': 443}],
     http_auth=awsauth,
     use_ssl=True,
     verify_certs=True,
     connection_class=RequestsHttpConnection,
     timeout=300
    )
    print (es_client.info())

    #search condition
    res3 = es_client.search(
        body={
            'size':10000,
            "query": {
                'bool':{
                    'filter':[
                        { "match": {'routeInfo.routeName' : routeName}},
                        { "match": {'routeInfo.direction' : direction}},
                        { "range": {'timestamp': {"gte":date}}}
                    ]
                }
            }
        }
    )

    #different format of face data
    number=[]
    for i in range(0,len(res3['hits']['hits'])):
        if( 'playerInfo' in res3['hits']['hits'][i]['_source'] ):
            if('ageRange' in res3['hits']['hits'][i]['_source']):
                a = []
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['routeName'])
                a.append(res3['hits']['hits'][i]['_source']['playerInfo']['plateNumber'])
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['position']['latitude'])
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['position']['longitude'])
                a.append(res3['hits']['hits'][i]['_source']['gender']['value'])
                a.append((res3['hits']['hits'][i]['_source']['ageRange']['low']+res3['hits']['hits'][i]['_source']['ageRange']['high'])/2)           
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['direction'])
                a.append(res3['hits']['hits'][i]['_source']['timestamp'])
                a.append(0)
                number.append(a)
            else:
                a = []
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['routeName'])
                a.append(res3['hits']['hits'][i]['_source']['playerInfo']['plateNumber'])
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['position']['latitude'])
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['position']['longitude'])
                a.append(res3['hits']['hits'][i]['_source']['gender']['value'])
                a.append(res3['hits']['hits'][i]['_source']['age'])           
                a.append(res3['hits']['hits'][i]['_source']['routeInfo']['direction'])
                a.append(res3['hits']['hits'][i]['_source']['timestamp'])
                a.append(0)
                number.append(a)
        else:
                continue

    for i in range(0,len(number)):
        temp = str(number[i][7]).split("T", 1)[1]
        number[i][8] = temp.split("+", 1)[0]
        number[i][7] = str(number[i][7]).split("T", 1)[0]
    print('data count: '+str(len(number)))
    
    return number

In [2]:
#臉部資料清理，刪除被重複拍到的臉部
#data must meet the following data rule
#csv file column
#0 : route number
#1 : player info(car number)
#2 : latitude
#3 : longtitude
#4 : gender
#5 : avgAge
#6 : direction
#7 : date
#8 : time

# argument: data as a matrix or dataframe , given time standard(in seconds), testNum(number)
# process: check if a particular row of data should be deleted
# output: return a dataframe in which unneeded data are omitted

import pandas as pd
import numpy as np
from operator import itemgetter
import sys

def cleanData(results):

    def timeDifference(time, i, j):
        difference = (time[i+j,1]-time[i,1])*3600 + (time[i+j,2]-time[i,2])*60 + time[i+j,3]-time[i,3]
        if(i==0):
            print(difference)
        return difference

    def ageDifference(inputData,i,j):
        difference = abs(inputData[i+j][5]-inputData[i][5])
        return difference

    #parameter
    #ageStandard: 幾歲以內視為同一人
    #timeStandard: 幾秒以內視為同一人
    #testNum: 幾筆以內視為同一人
    ageStandard = 1
    timeStandard = 1
    testNum = 10
    
    inputData = pd.DataFrame(results)
    inputData=inputData.sort_values(by=[0,1,7,8]) #sort data by 0 route number 1 player info 7 date 8 time（轉置）
    inputData=inputData.reset_index()
    arrayNum = len(inputData.index)
    inputData=inputData.T
    checkIfDelete = [0]*arrayNum # array is to see if given data is to be deleted
    time = np.zeros((arrayNum, 4))
    
    for i in range(0,arrayNum):#deal with time
        temp = inputData[i][8].split(':')
        time[i][1] = float(temp[0])#hour
        time[i][2] = float(temp[1])#minute
        time[i][3] = float(temp[2])#second

    for i in range(0,arrayNum-1):
        if(checkIfDelete[i]==1):
            continue
        if((i+testNum)>arrayNum-1):
            loopNum = arrayNum-1-i
           
        else:
            loopNum = testNum
        
        for j in range(1,loopNum+1):
            
            if(checkIfDelete[i+j]==1):
                continue
            if(inputData[i][1]!=inputData[i+j][1]):
                continue
            if(inputData[i][0]!=inputData[i+j][0]):
                continue
            if(inputData[i][6]!=inputData[i+j][6]):
                continue
            if(inputData[i][4]!=inputData[i+j][4]):
                continue
            if(inputData[i][7]!=inputData[i+j][7]):
                continue
            if(timeDifference(time,i,j)>timeStandard):
                continue
            if(ageDifference(inputData,i,j)>ageStandard):
                continue
            checkIfDelete[i+j] = 1

    deleteIndex=[]
    for i in range(0,arrayNum):
        if(checkIfDelete[i]==1):
            deleteIndex.append(i)
            
    resultData = inputData.T.drop(deleteIndex)
    resultData=resultData.reset_index()
    del resultData["level_0"]
    del resultData["index"]
    arrayNum2 = len(resultData.index)
    resultArray=[]
    for i in range(arrayNum2):
        resultArray.append(resultData.T[i].tolist())

        return resultArray


In [3]:
#face data preprocessing
#將上車的乘客資料整理成公車班次
#輸入：公車路線、方向、臉部資料
#輸出：
#stopName: 每趟班次的資訊，包括發車時間、日期、天氣、溫度
#stopTime: 每趟班次到各個站牌的到站時間
#rangeData: 每趟班次乘客上下車的情況
#rangeDataFull: 每趟班次各客群上下車的情況

import time, datetime, json
from openpyxl import load_workbook
import json

def preprocess(routeName, direction, data):

    #從MOTC取得的站牌資料
    #之前寫的時候發現用程式call需要MOTC的帳號不然會有unauthorized的錯誤
    #目前做法是找MOTC提供的json file(例如920:https://ptx.transportdata.tw/MOTC/v2/Bus/DisplayStopOfRoute/City/NewTaipei/920?$format=JSON)
    #存下來放進/MOTC的資料夾裡，檔名取路線名稱
    with open('./MOTC/'+routeName+'.json', 'r', encoding="utf-8") as f:
        stop = json.load(f)
    stopLat = []
    stopLon = []
    if direction == int(0):
        for i in stop[0]['Stops']:
            stopLat.append(i['StopPosition']['PositionLat'])
            stopLon.append(i['StopPosition']['PositionLon'])
    else:
        for i in stop[1]['Stops']:
            stopLat.append(i['StopPosition']['PositionLat'])
            stopLon.append(i['StopPosition']['PositionLon'])
            
    #取得天氣資料
    wb = load_workbook('weather.xlsx')
    timestamp = []
    temp = []
    rain = []
    for i in wb['0']['A'][1:]:
        a = str(i.value).split("T", 1)[0]
        b = str(i.value).split("T", 1)[1]
        c = b.split("+", 1)[0]
        timestamp.append(str(a+" "+c))
    for i in wb['0']['B'][1:]:
        temp.append(i.value)
    for i in wb['0']['C'][1:]:
        rain.append(i.value)
    
    #決定拍攝到的臉部資料屬於哪個站牌
    def pos(lat,lon,stopLat,stopLon):
        M = 1
        for i,j,k in zip(stopLat,stopLon,range(len(stopLat))):
            distance = pow((lat - i),2) + pow((lon - j),2)
            if(distance < M):
                stop = k
                M = distance
        return int(stop)

    #將天氣資料填入StopName中
    #0: sunny 1: rainy
    def weather(time,timestamp,temp,rain):
        for i in range(len(timestamp)):
            data = datetime.datetime.strptime(time,'%Y-%m-%d %H:%M:%S')
            cur = datetime.datetime.strptime(timestamp[i],'%Y-%m-%d %H:%M:%S')
            if data < cur:
                result = []
                if int(rain[i]) == 0 or int(rain[i]) == -99:
                    result.append(int(0))
                else:
                    result.append(int(1))
                result.append(int(temp[i]))
                return result

    #設定客群年齡區間
    def passenger(sex,age):
        if int(age)<20:
            if sex == 'male':
                return int(0)
            else:
                return int(6)
        elif int(age)<30 and int(age)>=20:
            if sex == 'male':
                return int(1)
            else:
                return int(7)
        elif int(age)<40 and int(age)>=30:
            if sex == 'male':
                return int(2)
            else:
                return int(8)    
        elif int(age)<50 and int(age)>=40:
            if sex == 'male':
                return int(3)
            else:
                return int(9)    
        elif int(age)<60 and int(age)>=50:
            if sex == 'male':
                return int(4)
            else:
                return int(10)
        else:
            if sex == 'male':
                return int(5)
            else:
                return int(11)
    
    #初始化
    #index of data: 6=direction,7=date,8=time
    segment = 12 #客群按年齡性別分成12類
    data = sorted(data,key = lambda x: (str(x[1]), str(x[7]),str(x[8])))
    stopName = np.empty((len(data),5),dtype='a32')
    stopTime = np.empty((len(data),len(stopLat)),dtype='a32')
    rangedDataFull = np.zeros((segment,len(data),len(stopLat)))

    rangeData = np.zeros((len(data),len(stopLat)))
    counter = -1

    #將臉部資料排班次、將經緯度轉換成站牌資料
    for i in range(0,len(data)):
        if i!=0:
            cur = datetime.datetime.strptime(data[i][8],'%H:%M:%S')
            pre = datetime.datetime.strptime(data[i-1][8],'%H:%M:%S')
        else:
            cur = datetime.datetime.strptime(data[i][8],'%H:%M:%S')
            pre = datetime.datetime.strptime(data[i][8],'%H:%M:%S')            
        if data[i][7]!=data[i-1][7] or data[i][6]!=data[i-1][6] or data[i][1]!=data[i-1][1] or (cur-pre).seconds>3600 or i == 0:
            counter = counter + 1
            now = 0
            stopName[counter][0] = str(data[i][7])+" "+str(data[i][8])
            stopName[counter][1] = str(data[i][6])
            w = weather(stopName[counter][0].decode("utf-8"),timestamp,temp,rain)
            stopName[counter][2] = w[0]
            stopName[counter][3] = w[1]       
        a = pos(data[i][2],data[i][3],stopLat,stopLon)
        rangeData[counter][a] = rangeData[counter][a]+1
        rangedDataFull[passenger(data[i][4],data[i][5])][counter][a] = rangedDataFull[passenger(data[i][4],data[i][5])][counter][a] + 1
        if(stopTime[counter][a].decode("utf-8") == '' and a >= now):
            stopTime[counter][a] = str(data[i][8])
            now = a

    #生成不同客群的上車資料
    less = []
    for i in range(len(rangeData)):
    #捨棄搭乘人數小於15人的班次
        if sum(rangeData[i,:]) <= 15:
            less.append(i) 
    rangeData = np.delete(rangeData, less, 0)
    stopTime = np.delete(stopTime, less, 0)
    stopName = np.delete(stopName, less, 0)
    r = np.zeros((segment,len(data)-len(less),len(stopLat)))
    for i in range(len(rangedDataFull)):
        r[i] = np.delete(rangedDataFull[i], less, 0)
    rangedDataFull = r

    #生成一趟公車在各站的到站時間
    for i in range(len(stopTime)):
        for j in range(len(stopTime[0])):
            if not stopTime[i,j].decode("utf-8") == '':
                start = datetime.datetime.strptime(stopTime[i,j].decode("utf-8"),'%H:%M:%S')
                startStop = j
                break
        for j in range(startStop,len(stopTime[0])):
            if stopTime[i,j].decode("utf-8") == '':
                continue
            cur = datetime.datetime.strptime(stopTime[i,j].decode("utf-8"),'%H:%M:%S')
            stopTime[i,j] = (cur - start).seconds + 30*startStop

        pre = ''
        for j in range(len(stopTime[0])):
            if pre == '' and stopTime[i,j].decode("utf-8") == '':
                continue
            elif pre != '' and stopTime[i,j].decode("utf-8") == '':
                nex = j
                while stopTime[i,nex].decode("utf-8") == '':
                    nex = nex + 1
                    if(nex == len(stopTime[0])):
                        break
                if nex == len(stopTime[0]):
                    break

                las = float(stopTime[i,nex])
                for k in range(j,nex):
                    stopTime[i,k] = float((k-j+1)*((las - pre)/(nex-j+1)) + pre)
                pre = las
                j = nex
            else:
                pre =float(stopTime[i,j])

    return rangedData, rangedDataFull, stopName, stopTime


In [56]:
#生成決策樹
#輸入：班次資訊、各班次乘客上下車情況
#輸出：決策模型(包括決策樹和與標籤對應的聚類資料)
from sklearn.cluster import KMeans
from sklearn.tree import tree

def decisionTree(rangedData,rangedDataFull,stopName,stopTime):

    #生成是否尖峰的變數 尖峰時段: 早上7~9點、下午5~7點
    def rush(day,hour):
        if day != datetime.datetime.strptime("2011-01-01",'%Y-%m-%d').weekday() and day != datetime.datetime.strptime("2011-01-02",'%Y-%m-%d').weekday():
            if (hour>=7 and hour<=9) or (hour>=17 and hour<=19): 
                return int(1) 
            else:
                return int(0)
        else:
            return int(0)

    #vector normalization
    for i in range(len(rangedData)):
        a = []
        for j in range(len(rangedData[0])):
            a.append(rangedData[i][j]/sum(rangedData[i]))
        rangedData[i] = a

    result = []
    for i in range(1,20):
         result.append(KMeans(n_clusters = i).fit(rangedData).inertia_)

    #嘗試找到最好的群數
    avg = result[0]/len(rangedData) #average within error
    for i in range(19):
        diff = result[i]-result[i+1]
        #threshold
        if diff<2*avg or i==18:
            bestClusNum = i+1
            break

    bestClus = KMeans(n_clusters = bestClusNum).fit(rangedData)
    print('best k: ',bestClusNum)
    treeX = np.zeros((len(stopName),5))
    treeY = np.zeros((len(stopName)))

    #生成決策樹
    for i in range(len(stopName)):
        treeX[i][0] = datetime.datetime.strptime(stopName[i,0].decode('utf-8'),'%Y-%m-%d %H:%M:%S').weekday()
        treeX[i][1] = int(str(stopName[i,0].decode('utf-8').split(' ')[1]).split(':')[0])
        treeX[i][2] = stopName[i,2]
        treeX[i][3] = stopName[i,3]
        treeX[i][4] = rush(treeX[i][0],treeX[i][1])
        treeY[i] = bestClus.labels_[i]
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(treeX, treeY)
    
    #生成標籤代表的聚類
    clusData = np.zeros((bestClusNum,len(rangedDataFull)+1,len(rangedData[0])))
    #生成乘客數據
    for i in range(bestClusNum):
        for j in range(len(rangedDataFull)):
            for k in range(len(rangedData[0])):
                r = [rangedDataFull[j][x][k] for x in range(len(stopName)) if treeY[x]==i]
                clusData[i][j][k] = sum(r)/len(r)
    #生成時間數據
    for i in range(bestClusNum):
        for j in range(len(rangedData[0])):
            r = [float(stopTime[x][j]) for x in range(len(stopName)) if treeY[x]==i and stopTime[x][j].decode("utf-8") != '']
            if len(r) != 0:
                clusData[i][len(rangedDataFull)][j] = sum(r)/len(r)
            else:
                clusData[i][len(rangedDataFull)][j] = 0
    #平滑時間區段
        pre = ''
        forecast = np.zeros(len(rangedData[0]))
        forecast = np.copy(clusData[i][len(rangedDataFull)])
        for j in range(len(rangedData[0])):
            if pre == '' and forecast[j] == 0:
                forecast[j] = j*30
            elif pre != '' and forecast[j] == 0:
                nex = j
                while forecast[nex] == 0:
                    nex = nex + 1
                    if(nex == len(forecast)):
                        for k in range(j,nex):
                            forecast[nex] = pre + (k+1)*30
                        break
                if nex == len(forecast):
                    break

                las = round(forecast[nex])
                for k in range(j,nex):
                    forecast[k] = round((k-j+1)*((las - pre)/(nex-j+1)) + pre)
                pre = las
                j = nex
            else:
                pre = round(forecast[j])
        for j in range(1,len(forecast)):
            if forecast[j] > forecast[j-1]:
                continue
            else:
                forecast[j] = forecast[j-1] +30
        clusData[i][len(rangedDataFull)] = np.copy(forecast)
        
    
    return clf,clusData

best k:  6
[   0.          126.19047619  221.83549784  275.35449735  332.31984127
  386.75453149  404.75582011  445.29168755  485.82653643  505.7104794
  557.85123883  620.42172485  672.9651906   723.59006979  783.90131343
  839.86901964  909.36516955  981.63488155 1038.65790043 1087.33028831
 1190.54507613 1271.28367347 1333.8765873  1415.95493651 1489.71161905
 1571.32604762 1633.40747619 1695.56690476 1777.283      1854.54766667
 1960.22066667 2049.21292517 2406.33006001 2596.80480438 2758.03235696
 2919.25990954 3066.86100245 3198.08683458 3329.07582462 3460.54924138
 3582.1016899  3687.98010076 3790.01857167 3887.62761316 4008.9900046
 4068.15955008 4221.27521471 4406.86562363 4537.27773435 4667.68984506
 4847.58729611 5028.45      ]
[   0.           71.38888889  132.01495726  161.0591716   192.01591252
  236.13535762  256.99206593  306.91898901  349.53132573  386.60186939
  429.32429029  485.65001077  535.68954009  589.56882487  645.84512425
  669.48987441  738.46510104  807.1903

(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
 array([[[3.92156863e-02, 0.00000000e+00, 0.00000000e+00, ...,
          1.96078431e-02, 0.00000000e+00, 0.00000000e+00],
         [1.37254902e-01, 3.92156863e-02, 0.00000000e+00, ...,
          1.96078431e-02, 0.00000000e+00, 5.88235294e-02],
         [2.15686275e-01, 0.00000000e+00, 0.00000000e+00, ...,
          3.92156863e-02, 1.96078431e-02, 7.84313725e-02],
         ...,
         [3.92156863e-02, 0.00000000e+00, 0.00000000e+00, ...,
          1.96078431e-02, 0.00000000e+00, 3.92156863e-02],
         [1.96078431e-02, 1.96078431e-02, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 3.92156863e-02],
         [0.0000000

In [33]:
stop

1.6666666666666667